## Imports and Configuration

In [8]:
# Imports
from splinter import Browser
from bs4 import BeautifulSoup as bs
import pandas as pd
import time
import numpy as np
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect

In [9]:
# Configure the splinter browser module
# Change this for a windows
#!which chromedriver
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [10]:
# Initialize browser object, open window
#browser = Browser('chrome', **ex_path)

In [11]:
browser.cookies.all()

{}

### Scrape

In [12]:
i=0
j=0
tickers = ['aapl', 'fb', 'goog', 'nflx']
calls_dataframes = {} # format => ticker: dataframe
puts_dataframes = {}  # format = ticker: dataframe

for symbol in tickers:
    print(f'--- Processing Scrape for: {symbol} ---')
    
    ticker_url = 'https://finance.yahoo.com/quote/'+symbol+'/options?p='+symbol
    print (ticker_url)
    browser.visit(ticker_url)
    ticker_html = browser.html
    ticker_scraper = bs(ticker_html, 'html.parser')
    
    time.sleep(3)
    # FIND CLOSE VALUE
    close_element = ticker_scraper.find('span', {'class': 'Trsdu(0.3s) Fw(b) Fz(36px) Mb(-4px) D(ib)'})
    ticker_close = close_element.get_text()
    
    time.sleep(3)
    
    # CALLS SECTION
    calls_table_element = ticker_scraper.find('table', {'class': 'calls table-bordered W(100%) Pos(r) Bd(0) Pt(0) list-options'})
    len(calls_table_element.contents)
    calls_table_element.contents[0]
    table_header = calls_table_element.contents[0]
    table_headers = table_header.findChild().findChildren()
    
    extracted_headers = []
    
    for index in range(0, len(table_headers), 2):
        extracted_headers.append(table_headers[index].get_text())
        
    extracted_headers.remove(extracted_headers[3])
    table_body = calls_table_element.contents[1]
    calls_rows = list(table_body.find_all('tr'))
        
    extracted_values = []

    for tr in calls_rows:
        row_data = []
        table_data = list(tr.find_all('td'))
        
        for td in table_data:
            row_data.append(td.get_text())
        extracted_values.append(row_data)
        
    ticker_calls_df = pd.DataFrame(columns = extracted_headers, data = extracted_values)
    ticker_calls_df['Ticker'] = symbol
    ticker_calls_df['C/P'] = 'C'

    ticker_calls_df['stock_price'] = ticker_close
    
    # Create the final calls dataframe that has data of all the tickers 
    calls_dataframes[symbol] = ticker_calls_df
    if i==0:
        main_calls_df = ticker_calls_df
        i=i+1
    else:
        main_calls_df = main_calls_df.append(ticker_calls_df)
        i=i+1
    ticker_calls_df = main_calls_df
    
    # CALLS SECTION (END)
    
    # Create the final output container that has all the dataframes
    calls_dataframes[symbol] = ticker_calls_df
    
    # PUTS SECTION starts
  
    puts_table_element = ticker_scraper.find('table', {'class': 'puts table-bordered W(100%) Pos(r) list-options'})
    len(puts_table_element.contents)
    puts_table_element.contents[0]
    table_header = puts_table_element.contents[0]
    table_headers = table_header.findChild().findChildren()
    
    extracted_headers = []
    
    for index in range(0, len(table_headers), 2):
        extracted_headers.append(table_headers[index].get_text())
        
    extracted_headers.remove(extracted_headers[3])
    table_body = puts_table_element.contents[1]
    puts_rows = list(table_body.find_all('tr'))
        
    extracted_values = []

    for tr in puts_rows:
        row_data = []
        table_data = list(tr.find_all('td'))
        
        for td in table_data:
            row_data.append(td.get_text())
        extracted_values.append(row_data)
        
    ticker_puts_df = pd.DataFrame(columns = extracted_headers, data = extracted_values)
    ticker_puts_df['Ticker'] = symbol
    ticker_puts_df['C/P'] = 'P'

    ticker_puts_df['stock_price'] = ticker_close
    
    # Create the final dataframe that has data of all the tickers 
    if j==0:
        main_puts_df = ticker_puts_df
        j=j+1
    else:
        main_puts_df = main_calls_df.append(ticker_puts_df)
        j=j+1
    ticker_puts_df = main_puts_df
    
    # PUTS SECTION (END)

    # Create the final output container that has all the dataframes
    puts_dataframes[symbol] = ticker_puts_df
    
    print(f'--- End Scrape for: {symbol} ---')

browser.quit()

--- Processing Scrape for: aapl ---
https://finance.yahoo.com/quote/aapl/options?p=aapl
--- End Scrape for: aapl ---
--- Processing Scrape for: fb ---
https://finance.yahoo.com/quote/fb/options?p=fb
--- End Scrape for: fb ---
--- Processing Scrape for: goog ---
https://finance.yahoo.com/quote/goog/options?p=goog
--- End Scrape for: goog ---
--- Processing Scrape for: nflx ---
https://finance.yahoo.com/quote/nflx/options?p=nflx
--- End Scrape for: nflx ---


In [29]:
ticker_calls_df = main_calls_df
ticker_puts_df = main_puts_df

In [14]:
print("Printing ticker_calls")
print("---------------------")
print(ticker_calls_df)

Printing ticker_calls
---------------------
          Contract Name         Last Trade Date  Strike Last Price    Bid  \
0   AAPL190927C00140000   2019-09-20 2:30PM EDT  140.00      79.50  79.25   
1   AAPL190927C00145000  2019-09-11 11:28AM EDT  145.00      75.85  74.20   
2   AAPL190927C00150000  2019-09-19 11:50AM EDT  150.00      72.22  69.25   
3   AAPL190927C00155000  2019-08-19 12:11AM EDT  155.00      52.53  65.80   
4   AAPL190927C00160000   2019-09-17 3:20PM EDT  160.00      60.10  59.20   
5   AAPL190927C00165000   2019-09-20 2:02PM EDT  165.00      54.40  54.25   
6   AAPL190927C00167500  2019-09-20 10:56AM EDT  167.50      54.53  51.75   
7   AAPL190927C00170000  2019-09-23 11:47AM EDT  170.00      49.05  49.25   
8   AAPL190927C00172500   2019-09-10 2:19PM EDT  172.50      42.00  46.75   
9   AAPL190927C00175000   2019-09-23 9:50AM EDT  175.00      44.45  44.20   
10  AAPL190927C00177500  2019-08-19 12:11AM EDT  177.50      27.90  43.25   
11  AAPL190927C00180000   2019-0

In [15]:
print("Printing ticker_puts")
print("---------------------")
print(ticker_puts_df)

Printing ticker_puts
---------------------
          Contract Name         Last Trade Date  Strike Last Price     Bid  \
0   AAPL190927C00140000   2019-09-20 2:30PM EDT  140.00      79.50   79.25   
1   AAPL190927C00145000  2019-09-11 11:28AM EDT  145.00      75.85   74.20   
2   AAPL190927C00150000  2019-09-19 11:50AM EDT  150.00      72.22   69.25   
3   AAPL190927C00155000  2019-08-19 12:11AM EDT  155.00      52.53   65.80   
4   AAPL190927C00160000   2019-09-17 3:20PM EDT  160.00      60.10   59.20   
5   AAPL190927C00165000   2019-09-20 2:02PM EDT  165.00      54.40   54.25   
6   AAPL190927C00167500  2019-09-20 10:56AM EDT  167.50      54.53   51.75   
7   AAPL190927C00170000  2019-09-23 11:47AM EDT  170.00      49.05   49.25   
8   AAPL190927C00172500   2019-09-10 2:19PM EDT  172.50      42.00   46.75   
9   AAPL190927C00175000   2019-09-23 9:50AM EDT  175.00      44.45   44.20   
10  AAPL190927C00177500  2019-08-19 12:11AM EDT  177.50      27.90   43.25   
11  AAPL190927C001800

In [16]:
#Remove the % sign from the Implied Volatility column before converting the dataframe into SQL
ticker_calls_df['Implied Volatility'] = ticker_calls_df['Implied Volatility'].str.replace('%', '')

In [17]:
# Rename all the column names that have spaces (replace Space with underscore)
ticker_calls_df.rename(columns={'Contract Name':'Contract_Name', 'Last Trade Date':'Last_Trade_Date', 'Last Price': 'Last_Price', 
                               '% Change':'Percent_Change', 'Open Interest':'Open_Interest', 
                               'Implied Volatility':'Implied_Volatility', 'C/P':'C_P'}, inplace=True)

In [18]:
# Change the index to column "Contract Name"
ticker_calls_df.set_index('Contract_Name', inplace=True)
ticker_calls_df.head(5)

,Last_Trade_Date,Strike,Last_Price,Bid,Ask,Change,Percent_Change,Volume,Open_Interest,Implied_Volatility,Ticker,C_P,stock_price
Contract_Name,,,,,,,,,,,,,
AAPL190927C00140000,2019-09-20 2:30PM EDT,140.00,79.50,79.25,79.55,0.00,-,10,40,189.45,aapl,C,219.35
AAPL190927C00145000,2019-09-11 11:28AM EDT,145.00,75.85,74.20,74.50,0.00,-,-,28,166.41,aapl,C,219.35
AAPL190927C00150000,2019-09-19 11:50AM EDT,150.00,72.22,69.25,69.55,0.00,-,10,91,163.28,aapl,C,219.35
AAPL190927C00155000,2019-08-19 12:11AM EDT,155.00,52.53,65.80,69.90,0.00,-,-,10,288.82,aapl,C,219.35
AAPL190927C00160000,2019-09-17 3:20PM EDT,160.00,60.10,59.20,59.50,0.00,-,2,11,130.47,aapl,C,219.35


In [19]:
# ticker_puts dataframe
#Remove the % sign from the Implied Volatility column before converting the dataframe into SQL
ticker_puts_df['Implied Volatility'] = ticker_puts_df['Implied Volatility'].str.replace('%', '')

In [20]:
# Rename all the column names that have spaces (replace Space with underscore)
ticker_puts_df.rename(columns={'Contract Name':'Contract_Name', 'Last Trade Date':'Last_Trade_Date', 'Last Price': 'Last_Price', 
                               '% Change':'Percent_Change', 'Open Interest':'Open_Interest', 
                               'Implied Volatility':'Implied_Volatility', 'C/P':'C_P'}, inplace=True)

In [21]:
# Change the index to column "Contract Name"
ticker_puts_df.set_index('Contract_Name', inplace=True)
ticker_puts_df.head(5)

,Last_Trade_Date,Strike,Last_Price,Bid,Ask,Change,Percent_Change,Volume,Open_Interest,Implied_Volatility,Ticker,C_P,stock_price
Contract_Name,,,,,,,,,,,,,
AAPL190927C00140000,2019-09-20 2:30PM EDT,140.00,79.50,79.25,79.55,0.00,-,10,40,189.45,aapl,C,219.35
AAPL190927C00145000,2019-09-11 11:28AM EDT,145.00,75.85,74.20,74.50,0.00,-,-,28,166.41,aapl,C,219.35
AAPL190927C00150000,2019-09-19 11:50AM EDT,150.00,72.22,69.25,69.55,0.00,-,10,91,163.28,aapl,C,219.35
AAPL190927C00155000,2019-08-19 12:11AM EDT,155.00,52.53,65.80,69.90,0.00,-,-,10,288.82,aapl,C,219.35
AAPL190927C00160000,2019-09-17 3:20PM EDT,160.00,60.10,59.20,59.50,0.00,-,2,11,130.47,aapl,C,219.35


In [22]:
# Import SQL Alchemy
from sqlalchemy import create_engine
# Create a database in SQLite if not already created - "yahoo_fin_db"
engine = create_engine("sqlite:///yahoo_fin_db.sqlite")
conn = engine.connect()

In [23]:
# Drop all the existing tables
engine.execute("DROP TABLE IF EXISTS ticker_calls_table")
engine.execute("DROP TABLE IF EXISTS Ticker_Calls_Table")
engine.execute("DROP TABLE IF EXISTS ticker_puts_table")
engine.execute("DROP TABLE IF EXISTS Ticker_Puts_Table")

In [24]:
#Check that there are no existing tables
engine.table_names()

[]

In [25]:
# Convert the calls dataframe to sql table
ticker_calls_df.to_sql(name='ticker_calls_table', con=engine, if_exists='append', index=True)
# Check if the table got created
engine.table_names()

['ticker_calls_table']

In [26]:
# Convert the puts dataframe to sql table
ticker_puts_df.to_sql(name='ticker_puts_table', con=engine, if_exists='append', index=True)
# Check if the table got created
engine.table_names()

['ticker_calls_table', 'ticker_puts_table']

In [27]:
# Execute the query to see all the records in the calls table
result_set = engine.execute("SELECT * From ticker_calls_table")
for record in result_set:
    print(record)

('AAPL190927C00140000', '2019-09-20 2:30PM EDT', '140.00', '79.50', '79.25', '79.55', '0.00', '-', '10', '40', '189.45', 'aapl', 'C', '219.35')
('AAPL190927C00145000', '2019-09-11 11:28AM EDT', '145.00', '75.85', '74.20', '74.50', '0.00', '-', '-', '28', '166.41', 'aapl', 'C', '219.35')
('AAPL190927C00150000', '2019-09-19 11:50AM EDT', '150.00', '72.22', '69.25', '69.55', '0.00', '-', '10', '91', '163.28', 'aapl', 'C', '219.35')
('AAPL190927C00155000', '2019-08-19 12:11AM EDT', '155.00', '52.53', '65.80', '69.90', '0.00', '-', '-', '10', '288.82', 'aapl', 'C', '219.35')
('AAPL190927C00160000', '2019-09-17 3:20PM EDT', '160.00', '60.10', '59.20', '59.50', '0.00', '-', '2', '11', '130.47', 'aapl', 'C', '219.35')
('AAPL190927C00165000', '2019-09-20 2:02PM EDT', '165.00', '54.40', '54.25', '54.55', '0.00', '-', '9', '25', '126.37', 'aapl', 'C', '219.35')
('AAPL190927C00167500', '2019-09-20 10:56AM EDT', '167.50', '54.53', '51.75', '52.05', '0.00', '-', '1', '7', '120.51', 'aapl', 'C', '219

In [28]:
# Execute the query to see all the records in the calls table
result_set = engine.execute("SELECT * From ticker_puts_table")
for record in result_set:
    print(record)

('AAPL190927C00140000', '2019-09-20 2:30PM EDT', '140.00', '79.50', '79.25', '79.55', '0.00', '-', '10', '40', '189.45', 'aapl', 'C', '219.35')
('AAPL190927C00145000', '2019-09-11 11:28AM EDT', '145.00', '75.85', '74.20', '74.50', '0.00', '-', '-', '28', '166.41', 'aapl', 'C', '219.35')
('AAPL190927C00150000', '2019-09-19 11:50AM EDT', '150.00', '72.22', '69.25', '69.55', '0.00', '-', '10', '91', '163.28', 'aapl', 'C', '219.35')
('AAPL190927C00155000', '2019-08-19 12:11AM EDT', '155.00', '52.53', '65.80', '69.90', '0.00', '-', '-', '10', '288.82', 'aapl', 'C', '219.35')
('AAPL190927C00160000', '2019-09-17 3:20PM EDT', '160.00', '60.10', '59.20', '59.50', '0.00', '-', '2', '11', '130.47', 'aapl', 'C', '219.35')
('AAPL190927C00165000', '2019-09-20 2:02PM EDT', '165.00', '54.40', '54.25', '54.55', '0.00', '-', '9', '25', '126.37', 'aapl', 'C', '219.35')
('AAPL190927C00167500', '2019-09-20 10:56AM EDT', '167.50', '54.53', '51.75', '52.05', '0.00', '-', '1', '7', '120.51', 'aapl', 'C', '219